# Merging DataFrames With Different Sizes and Missing Keys

In this notebook, we will walk through how to merge/join two DataFrames from a fictional class dataset. In this class, the professor has one DataFrame containing all of the students information called `students` and another that contains each students grades on the projects called `grades`. The only problem we have is that `students` has information for all of the students, but `grades` has multiple entries for most students, and none for others. Our goal is to combine these DataFrames so the professor can see the project grades for each student like in the table below. Then, we will do some additional analyses based on requests from the professor.

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>student_id</th>      <th>name</th>      <th>age</th>      <th>p1</th>      <th>p2</th>      <th>p3</th>    </tr>  </thead>  <tbody>    <tr>      <th>0</th>      <td>0</td>      <td>Eshan</td>      <td>21</td>      <td>71.435399</td>      <td>25.596660</td>      <td>99.330929</td>    </tr>    <tr>      <th>1</th>      <td>1</td>      <td>Alexis</td>      <td>20</td>      <td>40.069184</td>      <td>0.000000</td>      <td>0.000000</td>    </tr>    <tr>      <th>2</th>      <td>2</td>      <td>Torii</td>      <td>21</td>      <td>70.841185</td>      <td>62.279679</td>      <td>2.162617</td>    </tr>    <tr>      <th>3</th>      <td>3</td>      <td>Ben</td>      <td>21</td>      <td>0.000000</td>      <td>0.000000</td>      <td>0.000000</td>    </tr>    <tr>      <th>4</th>      <td>4</td>      <td>Zuri</td>      <td>21</td>      <td>7.710911</td>      <td>60.768933</td>      <td>5.149033</td>    </tr>    <tr>      <th>5</th>      <td>5</td>      <td>Amelia</td>      <td>21</td>      <td>13.318058</td>      <td>91.397381</td>      <td>0.000000</td>    </tr>    <tr>      <th>6</th>      <td>6</td>      <td>Elizabeth</td>      <td>19</td>      <td>0.000000</td>      <td>0.000000</td>      <td>0.000000</td>    </tr>    <tr>      <th>7</th>      <td>7</td>      <td>Jeremy</td>      <td>20</td>      <td>50.633919</td>      <td>40.719176</td>      <td>68.684132</td>    </tr>  </tbody></table>'

### Generating Random Data

Let's start by generating some random data. Below we generate the data for each of the DataFrames. You don't have to understand this code, though it certainly may be helpful in the future.

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Setting our seed so random values are consistent across runs
np.random.seed(320)

# Defining our lists for our 'grades' DataFrame
ids = [0, 0, 0, 1, 2, 2, 2, 4, 4, 4, 5, 5, 7, 7, 7]
projects = []
scores = list(np.random.random(len(ids)) * 100)

# Used to populate the 'projects' list.
p_count = 1
curr = 0
while curr < len(ids):
    if ids[curr - 1] != ids[curr]:
        p_count = 1
    projects.append(f"p{p_count}")
    p_count += 1
    curr += 1

# Creating a DataFrame out of our lists from above
grades = pd.DataFrame({
    "student_id" : ids,
    "project" : projects,
    "score" : scores,
})
grades

,student_id,project,score
0,0,p1,71.435399
1,0,p2,25.596660
2,0,p3,99.330929
3,1,p1,40.069184
4,2,p1,70.841185
5,2,p2,62.279679
6,2,p3,2.162617
7,4,p1,7.710911
8,4,p2,60.768933
9,4,p3,5.149033


In [3]:
# Defining our lists for our 'students' DataFrame
ids = list(range(8))
names = ["Eshan", "Alexis", "Torii", "Ben", "Zuri", "Amelia", "Elizabeth", "Jeremy"]
ages = list(np.random.randint(low = 18, high = 24, size = len(ids)))

# Creating a DataFrame out of our lists from above
students = pd.DataFrame({
    "student_id" : ids,
    "name" : names,
    "age" : ages,
})
students

,student_id,name,age
0,0,Eshan,21
1,1,Alexis,20
2,2,Torii,21
3,3,Ben,21
4,4,Zuri,21
5,5,Amelia,21
6,6,Elizabeth,19
7,7,Jeremy,20


### Merging the DataFrames

Before we merge the DataFrames, we need to review (or learn) a concept that was likely (or likely will be covered) in STAT 240 -- pivoting. In this case, we will be using pandas's `DataFrame.pivot_table()` function to reshape our `grades` DataFrame (see the documentation [here](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.pivot_table.html) and some more information on it [here](https://www.w3resource.com/pandas/dataframe/dataframe-pivot.php#:~:text=The%20pivot()%20function%20is,a%20MultiIndex%20in%20the%20columns.)). `pivot_table()` only has one *required* argument, but we will be giving it four arguments: `data`, `index`, `columns`, and `values`. Let's look at each of those a little closer:

- `data`: This one is pretty self-explanatory. `data` is simply the DataFrame weare trying to reshape.
- `index`: This is the column in `data` that we are trying to aggregate. In other words, these values will be the new index of our pivoted data. In our example, this will be "student_id" because we are trying to get the project data for each student based on their unique student id.
- `columns`: This is the column that we want to use to make our new columns in the pivoted data. In our example, this will be "project" because we want to get a column for each project that was assigned in this semester.
- `values`: This is the column that will be used to populate the new cells in our pivoted data. In our case, we will use "scores" because we want each resulting cell to describe the score asscoaited with each project and student id pair.

Note that we have some missing values, but we will take care of that later.

In [4]:
# Creating our pivoted DataFrame based on the information above
grades_pivoted = grades.pivot_table(index = "student_id", columns = "project", values = "score")
grades_pivoted

project,p1,p2,p3
student_id,,,
0,71.435399,25.596660,99.330929
1,40.069184,NaN,NaN
2,70.841185,62.279679,2.162617
4,7.710911,60.768933,5.149033
5,13.318058,91.397381,NaN
7,50.633919,40.719176,68.684132


Our last step in adding these new columns is to use the `pandas.merge()` function (documentation [here](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html)). This function allows us to merge two data frames using a "join". Remeber that there are several different joins we can use, and you can read more about them in the documentation. You may have noticed that the `grades_pivoted` and `grades` DataFrames were both missing students. This could because they didn't complete projects, didn't turn projects in, or any other reason. But if we try to do an inner join, we will be missing values (see for yourself below. Try changing "outer" to be "inner"). In our case, we will be using an outer join, which will make sure that the student ids from each DataFrame are used for joining, and they will thus be present in the resulting DataFrame.

In [5]:
# Merging our DataFrames on the unique student id of each student
merged_df = pd.merge(students, grades_pivoted, how = "outer", on = "student_id")
merged_df = merged_df.fillna(0) # Fill in any missing project with a 0
merged_df

,student_id,name,age,p1,p2,p3
0,0,Eshan,21,71.435399,25.596660,99.330929
1,1,Alexis,20,40.069184,0.000000,0.000000
2,2,Torii,21,70.841185,62.279679,2.162617
3,3,Ben,21,0.000000,0.000000,0.000000
4,4,Zuri,21,7.710911,60.768933,5.149033
5,5,Amelia,21,13.318058,91.397381,0.000000
6,6,Elizabeth,19,0.000000,0.000000,0.000000
7,7,Jeremy,20,50.633919,40.719176,68.684132


We have now combined our two DataFrames in a way that allows us to see that project statistics for each student. Now that it's the end of the semester, we are going to get the mean score for each student's projects grade and add it to a column. Make sure in this case we specify `axis = 1` so that we are getting the mean for each student and not for each project.

In [6]:
merged_df["mean"] = merged_df[["p1", "p2", "p3"]].mean(axis=1)
merged_df

,student_id,name,age,p1,p2,p3,mean
0,0,Eshan,21,71.435399,25.596660,99.330929,65.454329
1,1,Alexis,20,40.069184,0.000000,0.000000,13.356395
2,2,Torii,21,70.841185,62.279679,2.162617,45.094494
3,3,Ben,21,0.000000,0.000000,0.000000,0.000000
4,4,Zuri,21,7.710911,60.768933,5.149033,24.542959
5,5,Amelia,21,13.318058,91.397381,0.000000,34.905146
6,6,Elizabeth,19,0.000000,0.000000,0.000000,0.000000
7,7,Jeremy,20,50.633919,40.719176,68.684132,53.345742


Now it's your turn to add another statistic. The professor wants to see what the range was between each student's highest score and lowest score on projects. Add a column called "p_range" that contains this range value for each student. You may consider adding columns "high_score" and "low_score" to help you.

In [7]:
# Your code goes here

# Creating the "high_score" column

# Creating the "low_score" column

# Creating the "p_range" column


<details>
    <summary>Correct Answer</summary>
    '<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>p_range</th>    </tr>  </thead>  <tbody>    <tr>      <th>0</th>      <td>73.734269</td>    </tr>    <tr>      <th>1</th>      <td>40.069184</td>    </tr>    <tr>      <th>2</th>      <td>68.678568</td>    </tr>    <tr>      <th>3</th>      <td>0.000000</td>    </tr>    <tr>      <th>4</th>      <td>55.619900</td>    </tr>    <tr>      <th>5</th>      <td>91.397381</td>    </tr>    <tr>      <th>6</th>      <td>0.000000</td>    </tr>    <tr>      <th>7</th>      <td>27.964955</td>    </tr>  </tbody></table>'
</details>

<details>
    <summary>Code</summary>
    <pre><code>
    merged_df["low_score"] = merged_df[["p1", "p2", "p3"]].min(axis = 1)
    merged_df["high_score"] = merged_df[["p1", "p2", "p3"]].max(axis = 1)
    merged_df["p_range"] = merged_df["high_score"] - merged_df["low_score"]
    merged_df
    </code></pre>
</details>